In [ ]:
log_path = "backtests/2024-04-13_00-41-33.log"

In [45]:
import re
from io import StringIO

import pandas as pd
import plotly.express as px


def read_file_sections(filepath):
    section_delimiters = ["Sandbox logs:", "Activities log:", "Trade History:"]
    current_section = None
    data = {key: [] for key in section_delimiters}

    with open(filepath, "r") as file:
        for line in file:
            line = line.strip()
            if line in section_delimiters:
                current_section = line
                continue
            if current_section:
                data[current_section].append(line)

    return data


def extract_trades(text):
    # Regex to find all blocks enclosed in curly braces
    trade_pattern = r"\{[^{}]*\}"
    # Regex to extract key-value pairs within curly braces
    kv_pattern = r"\"(\w+)\":\s*(\"[^\"]*\"|\d+)"

    trades = []
    matches = re.findall(trade_pattern, text)
    for match in matches:
        # Extract key-value pairs from each match
        trade_data = {}
        for key, value in re.findall(kv_pattern, match):
            # Remove quotes from strings
            if value.startswith('"') and value.endswith('"'):
                value = value.strip('"')
            else:
                value = int(value)  # Convert numeric values to integers
            trade_data[key] = value
        trades.append(trade_data)
    return trades


sections = read_file_sections(log_path)
activities_data = "\n".join(sections["Activities log:"])
activities_df = pd.read_csv(StringIO(activities_data), delimiter=";")
trade_history_text = "\n".join(sections["Trade History:"])
trades = extract_trades(trade_history_text)
trade_history_df = pd.DataFrame(trades)

In [46]:
activities_df

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-2,0,AMETHYSTS,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,0.0
1,-2,0,STARFRUIT,5002,1,4997.0,31.0,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,5002.5,0.0
2,-2,100,AMETHYSTS,9996,2,9995.0,22.0,NaN,NaN,10004,2,10005.0,22.0,NaN,NaN,10000.0,2.0
3,-2,100,STARFRUIT,4997,24,NaN,NaN,NaN,NaN,5003,2,5004.0,22.0,NaN,NaN,5000.0,-55.0
4,-2,200,AMETHYSTS,9995,20,NaN,NaN,NaN,NaN,10005,20,NaN,NaN,NaN,NaN,10000.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-2,199700,STARFRUIT,4949,27,NaN,NaN,NaN,NaN,4956,27,NaN,NaN,NaN,NaN,4952.5,1896.5
3996,-2,199800,AMETHYSTS,9995,21,NaN,NaN,NaN,NaN,10002,8,10005.0,21.0,NaN,NaN,9998.5,2263.5
3997,-2,199800,STARFRUIT,4949,21,NaN,NaN,NaN,NaN,4956,21,NaN,NaN,NaN,NaN,4952.5,1896.5
3998,-2,199900,AMETHYSTS,9996,2,9995.0,30.0,NaN,NaN,10004,2,10005.0,30.0,NaN,NaN,10000.0,2262.0


In [47]:
trade_history_df

,timestamp,buyer,seller,symbol,currency,price,quantity
0,0,,SUBMISSION,AMETHYSTS,SEASHELLS,10002,1
1,0,,SUBMISSION,STARFRUIT,SEASHELLS,5002,1
2,0,,SUBMISSION,STARFRUIT,SEASHELLS,4997,19
3,0,,,STARFRUIT,SEASHELLS,5003,13
4,0,,,STARFRUIT,SEASHELLS,5003,1
...,...,...,...,...,...,...,...
1764,199400,,,STARFRUIT,SEASHELLS,4950,1
1765,199500,,SUBMISSION,AMETHYSTS,SEASHELLS,10002,1
1766,199500,,,AMETHYSTS,SEASHELLS,10002,1
1767,199600,,SUBMISSION,STARFRUIT,SEASHELLS,4956,2


In [50]:
def plot_price_trends(dataframe):
    fig = px.line(dataframe, x='timestamp', y='price', color='symbol', 
                  title='Price Trends Over Time', 
                  labels={'price': 'Trade Price', 'timestamp': 'Timestamp'})
    fig.update_layout(xaxis_title="Timestamp", yaxis_title="Price", legend_title="Product")
    return fig

plot_price_trends(trade_history_df).show()


# # Volume of Trades Over Time
# fig_volume_trades = px.line(trade_history_df, x='timestamp', y='quantity', color='symbol',
#                             title='Volume of Trades Over Time', labels={'quantity': 'Trade Quantity', 'timestamp': 'Timestamp'})
# fig_volume_trades.show()
